# Projeto 7: Classificação multiclasse Iris com validação cruzada

## Etapa 1: Importação das bibliotecas

In [1]:
!pip install skorch==0.7.0

     |████████████████████████████████| 112kB 6.5MB/s 


In [2]:
!pip install torch==1.4.0

     |████████████████████████████████| 753.4MB 21kB/s 
ERROR: torchvision 0.8.1+cu101 has requirement torch==1.7.0, but you'll have torch 1.4.0 which is incompatible.
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101


In [16]:
import pandas as pd
import numpy as np
import seaborn as sns
import torch
import torch.nn.functional as F

from torch import nn, optim
from skorch import NeuralNetClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder

## Etapa 2: Base de dados

In [6]:
np.random.seed(123)
torch.manual_seed(123)

In [8]:
df = pd.read_csv('/content/iris.csv')
X = df.iloc[:,0:4].values
y = df.iloc[:,4].values
X.shape, y.shape

((150, 4), (150,))

In [9]:
np.unique(df['class'], return_counts=True)

(array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object),
 array([50, 50, 50]))

In [10]:
encoder = LabelEncoder()
y = encoder.fit_transform(y)

In [11]:
np.unique(y, return_counts=True)

(array([0, 1, 2]), array([50, 50, 50]))

In [12]:
X = X.astype('float32')
y = y.astype('int64')

## Etapa 3: Construção do modelo

In [13]:
class cls_torch(nn.Module):
  def __init__(self):
    super().__init__()

    self.dense0 = nn.Linear(4, 16)
    self.activation0 = nn.ReLU()

    self.dense1 = nn.Linear(16, 16)
    self.activation1 = nn.ReLU()

    self.dense2 = nn.Linear(16, 3)

  def forward(self, X):
    
    X = self.dense0(X)
    X = self.activation0(X)
    
    X = self.dense1(X)
    X = self.activation1(X)

    X = self.dense2(X)
    return X

In [14]:
cls_sklearn = NeuralNetClassifier(module=cls_torch,
                                  criterion = torch.nn.CrossEntropyLoss,
                                  optimizer = torch.optim.Adam,
                                  max_epochs = 1000,
                                  batch_size = 10,
                                  train_split = False)

## Etapa 4: Validação cruzada

In [17]:
results = cross_val_score(cls_sklearn, X, y, cv = 5, scoring = 'accuracy')

Streaming output truncated to the last 5000 lines.
      9        0.5162  0.0301
     10        0.4109  0.0215
     11        0.5542  0.0245
     12        0.4098  0.0235
     13        0.5332  0.0239
     14        0.4993  0.0228
     15        0.3979  0.0215
     16        0.3369  0.0286
     17        0.3687  0.0257
     18        0.3944  0.0225
     19        0.2903  0.0195
     20        0.3097  0.0220
     21        0.2867  0.0183
     22        0.2556  0.0223
     23        0.2571  0.0212
     24        0.2474  0.0281
     25        0.2309  0.0231
     26        0.2160  0.0212
     27        0.2134  0.0219
     28        0.2090  0.0204
     29        0.2031  0.0245
     30        0.1998  0.0222
     31        0.1923  0.0220
     32        0.1882  0.0202
     33        0.1818  0.0223
     34        0.1826  0.0256
     35        0.1794  0.0218
     36        0.1772  0.0185
     37        0.1729  0.0211
     38        0.1697  0.0223
     39        0.1644  0.0198
     40        0.16

In [19]:
mean = results.mean()
std = results.std()

In [20]:
mean

0.9800000000000001

In [21]:
std

0.02666666666666666